In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter

# 기존 PDF 파일 로드
input_file = r"C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\240120과제.pdf"
output_file = "과제/241020/data.pdf"  # 파일 확장자 포함

# 디렉터리 생성
output_dir = os.path.dirname(output_file)  # 디렉터리 경로 추출
if not os.path.exists(output_dir):
    os.makedirs(output_dir)  # 디렉터리가 없으면 생성

# PDF 읽기 및 작성
reader = PdfReader(input_file)
writer = PdfWriter()

# 여러 페이지 추가
pages_to_extract = [0, 1, 2]
for page_num in pages_to_extract:
    writer.add_page(reader.pages[page_num])

# 추출된 PDF 저장
with open(output_file, "wb") as f:
    writer.write(f)

print(f"페이지 {', '.join(str(p + 1) for p in pages_to_extract)}이 {output_file}로 저장되었습니다.")

In [1]:
FILE_PATH = r"C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\data.pdf"

In [2]:
def show_metadata(docs):
    if docs:
        print("[metadata]")
        print(list(docs[0].metadata.keys()))
        print("\n[examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")

In [13]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[1].page_content[:500])

MAP
Instruction Data
Q: Why does MATLAB's pwelch function use 2k 
degrees of freedom for confidence intervals in 
power spectral density estimation, and what is 
the reasoning behind this choice?
A: In the calculation of confidence intervals for 
power spectral density (PSD) estimation using 
Welch‘s method, MATLAB’s pwelch function 
uses 2k degrees of freedom instead of k-1 or 2k-
1. This choice is based on ……
Original SFT Dataset
Combined Set
Pretraining Corpus
Difference Set
Rewritten Set
Rew


In [14]:
show_metadata(docs)

[metadata]
['source', 'file_path', 'page', 'total_pages', 'format', 'title', 'author', 'subject', 'keywords', 'creator', 'producer', 'creationDate', 'modDate', 'trapped']

[examples]
source       : C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\data.pdf
file_path    : C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\data.pdf
page         : 0
total_pages  : 3
format       : PDF 1.5
title        : 
author       : 
subject      : 
keywords     : 
creator      : 
producer     : PyPDF2
creationDate : 
modDate      : 
trapped      : 


In [21]:
import os

os.environ['OPENAI_API_KEY']=''
os.environ['LANGCHAIN_TRACING_V2']=''
os.environ['LANGCHAIN_ENDPOINT']=''
os.environ['LANGCHAIN_API_KEY']=''
os.environ['LANGCHAIN_PROJECT']=''

In [16]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
import json

# 1. PDF Loading
def load_pdf(file_path):
    """
    Load the text from a PDF file.
    
    :param file_path: Path to the PDF file.
    :return: Extracted text from the PDF.
    """
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# 2. Splitting the Text
def split_text(text, chunk_size=500, chunk_overlap=50):
    """
    Split the text into chunks.
    
    :param text: The full text to split.
    :param chunk_size: Maximum size of each chunk.
    :param chunk_overlap: Overlap size between chunks.
    :return: List of text chunks.
    """
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_text(text)

# 3. Embedding the Text
def embed_text(chunks):
    """
    Generate embeddings for the text chunks using HuggingFaceEmbeddings.
    
    :param chunks: List of text chunks.
    :return: A FAISS vectorstore containing the embeddings.
    """
    embeddings = HuggingFaceEmbeddings()  # Default model: all-MiniLM-L6-v2
    vectorstore = FAISS.from_texts(chunks, embeddings)
    return vectorstore

# 4. Generate Questions Using GPT
def generate_questions(text, num_questions=10):
    """
    Generate a list of questions based on the provided text using GPT.
    
    :param text: The input text for question generation.
    :param num_questions: Number of questions to generate.
    :return: List of questions.
    """
    llm = OpenAI(temperature=0.7)
    prompt = f"Generate {num_questions} questions based on the following text:\n{text}"
    response = llm(prompt)
    return response.split("\n")

# 5. Find the Most Similar Embedding
def find_most_similar(vectorstore, questions):
    """
    Find the most similar embedding for each question.
    
    :param vectorstore: FAISS vectorstore with text embeddings.
    :param questions: List of questions.
    :return: Dictionary mapping questions to their most similar answers.
    """
    results = {}
    for question in questions:
        docs = vectorstore.similarity_search(question, k=1)
        results[question] = docs[0].page_content if docs else "No matching content found."
    return results

# 6. Save Results to JSON
def save_to_json(data, file_path):
    """
    Save the data to a JSON file.
    
    :param data: Data to save.
    :param file_path: Path to the JSON file.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [24]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
import json

# 1. PDF Loading
def load_pdf(file_path):
    """
    Load the text from a PDF file.
    
    :param file_path: Path to the PDF file.
    :return: Extracted text from the PDF.
    """
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# 2. Splitting the Text
def split_text(text, chunk_size=500, chunk_overlap=50):
    """
    Split the text into chunks.
    
    :param text: The full text to split.
    :param chunk_size: Maximum size of each chunk.
    :param chunk_overlap: Overlap size between chunks.
    :return: List of text chunks.
    """
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_text(text)

# 3. Embedding the Text
def embed_text(chunks):
    """
    Generate embeddings for the text chunks using HuggingFaceEmbeddings.
    
    :param chunks: List of text chunks.
    :return: A FAISS vectorstore containing the embeddings.
    """
    embeddings = HuggingFaceEmbeddings()  # Default model: all-MiniLM-L6-v2
    vectorstore = FAISS.from_texts(chunks, embeddings)
    return vectorstore

# 4. Generate Questions Using GPT
def generate_questions(text, num_questions=10):
    """
    Generate a list of questions based on the provided text using GPT.
    
    :param text: The input text for question generation.
    :param num_questions: Number of questions to generate.
    :return: List of questions.
    """
    llm = OpenAI(temperature=0.7)
    prompt = f"Generate {num_questions} questions based on the following text:\n{text}"
    response = llm(prompt)
    return response.split("\n")

# 5. Find the Most Similar Embedding
def find_most_similar(vectorstore, questions):
    """
    Find the most similar embedding for each question.
    
    :param vectorstore: FAISS vectorstore with text embeddings.
    :param questions: List of questions.
    :return: Dictionary mapping questions to their most similar answers.
    """
    results = {}
    for question in questions:
        docs = vectorstore.similarity_search(question, k=1)
        results[question] = docs[0].page_content if docs else "No matching content found."
    return results

# 6. Save Results to JSON
def save_to_json(data, file_path):
    """
    Save the data to a JSON file.
    
    :param data: Data to save.
    :param file_path: Path to the JSON file.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Main Workflow Example
if __name__ == "__main__":
    pdf_path = r"C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\data.pdf"  # Replace with the actual file path
    json_output_path = r"C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\output.json"  # Replace with the desired output path

    # Step 1: Load the PDF
    text = load_pdf(pdf_path)

    # Step 2: Split the text
    chunks = split_text(text)

    # Step 3: Embed the text
    vectorstore = embed_text(chunks)

    # Step 4: Generate questions
    questions = generate_questions(text, num_questions=10)

    # Step 5: Find the most similar embeddings
    results = find_most_similar(vectorstore, questions)

    # Step 6: Save results to JSON
    save_to_json(results, json_output_path)

    print(f"Questions and answers saved to {json_output_path}")

C:\Users\user\AppData\Local\Temp\ipykernel_23556\2636064405.py:48: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()  # Default model: all-MiniLM-L6-v2


Questions and answers saved to C:\Users\user\Desktop\Github\JJU1\07_황보선\수업\5주차\output.json
